# Table of contents

1. [Introduction](#Introduction) <br>
2. [Imports](#imports)<br>
3. [Data Acquisition](#data_acquisition)<br>
   3.1 [Scraping Challanges](#scraping_challanges)<br>
   3.2 [Util Functions](#util_functions)<br>
   3.3 [Scraping Process](#scraping_process)<br>
   3.4 [Crawling](#crawling)<br>
   3.5 [Data Scraped](#data_scraped)<br>
4. [Data Cleaning](#data_cleaning)<br>
   4.1 [Data NA to Real NA](#data_na_to_real_na)<br>
   4.2 [Cleaning and Converting](#cleaning_and_converting)<br>
   4.3 [Converting to Categorical](#categorical_conversion)<br>
   4.4 [Removing Duplicates](#removing_duplicates)<br>
   4.5 [Filling in missing values](#filling_in_missing_values)<br>
5. [EDA](#eda)<br>
   5.1 [Numeric fields Correlation to Price](#numeric_correlation)<br>
   5.2 [Ram Capacity distribution](#ram_capacity_distribution)<br>
   5.3 [Brands Distribution](#brands_distribution)<br>
   5.4 [Mean price by brand](#mean_price_by_brand_)<br>
   5.5 [Screen size distribution](#screen_size_distribution)<br>
   5.6 [Mean price by screen size](#mean_price_by_screen_size)<br>
   5.7 [Mean price by GPU](#mean_price_by_gpu)<br>
   5.8 [Mean price by GPU](#mean_price_by_series)<br>
6. [Machine Learning](#machine_learning_prep)<br>
   6.1 [Classification](#classification)<br>
   6.2 [Helper Functions](#ml_helper_functions)<br>
   6.3 [Predictions](#predictions)<br>
7. [Conclusion](#conclusion)<br>


<a name='introduction'></a>
# Introduction
<p>
    Today we have so many phones in the market<br>
    All with different prices, brands, graphic cards, hard drives types and other specs.<br>
    <br>
    we thought to our selves what affects the price the most. is it the brand? is it the camera quality? or is it a simple thing like the type of RAM or something else? 
    <br>
    Also, could we predict the price of a phones based on it's specs? which elements are critical for that prediction?
    <br>
    To answer the above questions we set a goal to our research to see if we can build a model that will predict the price of a phones based of it's specs and also to figure out along the way which elements of the specs affect the price the most.
</p>

<a id='imports'></a>
<h2 id="imports">Imports</h2>

In [1]:
from bs4 import BeautifulSoup
import requests
import re
import pandas as pd
from random import randint
import time
from time import sleep
from typing import List
from sklearn.preprocessing import MinMaxScaler
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib import gridspec
from sklearn import linear_model, metrics, preprocessing
from sklearn.preprocessing import StandardScaler, MinMaxScaler, LabelEncoder
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.metrics import r2_score, f1_score

<a id="data_acquisition"></a>
<h2>Data Acquisition</h2>
<p>Considering our options of data acquisition sources, we decided to look for the biggest electronics comparsion websites and scrape data which we thought will be helpful and save it as a dataframe.

The top options we found were:</p>
<ul>
   <li> <a href="https://zap.com/"target="_blank">Zap</a></li>
    <li><a href="https://amazon.com/"target="_blank">Amazon</a></li>
    <li><a href="https://www.ret.co.il/"target="_blank">Ret</a></li>
    <li><a href="https://www.gadgetsnow.com/"target="_blank">gadgetsnow</a></li>
    <li><a href="https://www.wisebuy.co.il/"target="_blank">wisebuy</a></li>
</ul>

<p>After a punctual search in these websites for relevant data and great community scale, we have decided to move on with Zap And Ret due to the popularity of the sites and the amount of features provided for each phones.</p>




<a id='scraping_challanges'></a>
<h3>Scraping challanges</h3>
<p>On our first attempts to scrape the data from Zap and Ret we incounterd the following issues:<br>
    <ul>
       <li> When sending too many requests in a short period of time, the websites started to slow down and the scraping took a long period of time.</li>
        <li>The default user agent used by requests was python: "requests/2.26.0" and we were blocked by Zap and Ret after a few requests.</li>
     </ul>
</p>
<p> The way we dealed with those issues was by using sleep and a custom header.

<a id='util_functions'></a>

<h3>
Some util functions to help scrape the data:
</h3>
<p>
    <ul>
        <li>sleepms- a function made in order to create a delay so the amount of requests sends to the server in parralel will not affect the scraping performance and other users experience in the website</li>
        <li>random_wait- a function which is used in order to create random delay so our scraping methodology will not be detected by the servers.</li>
       <li> get_fake_user_agent- a function which provides us with a non suspectable user agent.</li>

In [2]:
def sleepms(milliseconds: int):
    seconds = 0.001 * milliseconds
    sleep(seconds)
    
def random_wait():
    sleepms(randint(1, 250))

def get_fake_user_agent():
    # phone user agent
    return {"User-Agent":"Mozilla/5.0 (Linux; Android 6.0; Nexus 5 Build/MRA58N) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/111.0.0.0 Mobile Safari/537.36"}

<a id='scraping_process'></a>
<h3>Scraping Process</h3>
<p>
We went into the website and started exploring in order to find the best route for our scraping proccess.
</p>
<p>
We could clearly see that our best option to get maximum data was in the extra details page which contains huge amount of data.<br>
</p>

</p>
At this point we asked ourselves what is the exact data we wish to scrape?<br>

we decided to go with the following and rename them for the rest of the process:
<ul>
    <li>יצרן - Brand</li>
    <li>תאריך כניסה לאתר - Enter Date</li>
    <li>שנת הכרזה – Out date</li>
    <li>מערכת הפעלה - Operating System</li>
    <li>משקל - Weight</li>
    <li>סדרה – Series</li>
    <li>טלפון חכם-SmartPhone</li>
    <li>נפח זיכרון RAM - RAM Capacity</li>
    <li>מהירות מעבד - CPU speed</li>
    <li>דור- generation</li>
    <li>גודל מסך - Screen Size</li>
    <li>מספר ליבות – number of cores</li>
    <li>רזולוצית מסך - Screen Resolution</li>
    <li>צפיפות פיקסלים  - pixels</li>
    <li>סוג מסך - Screen Type</li>
    <li>קצב ריענון תצוגה - FPS</li>
    <li>עמידות במים – Water resistance</li>
    <li>צבע – colors</li>
    <li>ממשק הפעלה - Operating interface</li>
    <li>כרטיס זיכרון - GPU</li>
    <li>מצלמה קדמית – Front Camera</li>
    <li>מצלמה–Camera</li>
    <li>קורא טביעת אצבע - Fingerprint reader</li>
    <li>רשת אלחוטית - Wifi modal</li>
    <li>מחיר מינימלי - Min price</li>
    <li>מחיר מקסימלי - Max price</li>
    <li>קורא כרטיסים – NFC</li>
    <li>Bluetooth</li>
    <li>תמיכה ב esim</li>
    <li>תמיכה בטעינה אלחוטית - Wireless charging support</li>
    <li>קיבולת סוללה - Battery capacity</li>
</ul>
Also, we decided to save the page number in which Zap decided to place the phones, we figured it has a meaning we could use:
<ul>
    <li>Page Number</li>
</ul>


```python
# ZAP
def zap_crawl_data(BASE_URL,phone_id):
    price_detail = requests.get(f'{BASE_URL}//model.aspx?modelid={phone_id}',headers = get_fake_user_agent())
    price_soup = BeautifulSoup(price_detail.text, 'html.parser')
    spans = price_soup.find('div',attrs={'class':'prices-txt'}).find_all('span',limit=None)
    titles = ['מחיר מינימלי','מחיר מקסימלי']
    if len(spans) == 3:
        values = [ spans[1].text, spans[0].text ]
    elif len(spans) == 2: 
        values = [ spans[0].text, spans[0].text ]
    else:
        return None


    phone_detail = requests.get(f'{BASE_URL}//compmodels.aspx?modelid={phone_id}',headers = get_fake_user_agent())
    phone_soup = BeautifulSoup(phone_detail.text, 'html.parser').find('div',attrs={'class':'compareBox'})
    links = phone_soup.find_all('div', attrs={'class' : 'paramRow'},limit=None)
    
    for i in range(0, len(links)):
        titles.append(links[i].find('div',attrs={'class' : 'ParamCol'}).text.replace('?','').strip())
        values.append(links[i].find('div',attrs={'class' : 'ParamColValue'}).text.strip())

    return dict(zip(titles,values))

# RET
def ret_crawl_data(BASE_URL,phone_id):
    url = f'{BASE_URL}m/{phone_id}/specs'
    page = requests.get(url,headers = get_fake_user_agent())
    soup = BeautifulSoup(page.text, 'html.parser')
    meta = soup.find('div',attrs={'class','price'})
    lowPrice = meta.find('meta',attrs={'itemprop':'lowPrice'})
    highPrice = meta.find('meta',attrs={'itemprop':'highPrice'})
    if(lowPrice == None or highPrice == None):
        return None
    titles = ['מחיר מינימלי','מחיר מקסימלי']
    values = [lowPrice.get('content'),highPrice.get('content')]
    properties = soup.find('div',attrs={'class','single-product-tab'}).find_all('ul',attrs={'class':'list-group list-group-horizontal'},limit=None)
    for i in range(0,len(properties)):
        [title,value] = properties[i].find_all('li',limit=None)
        titles.append(title.text.strip())
        values.append(value.text.strip()) 
    
    return dict(zip(titles,values))
```

In [3]:
def translate_names(df):
    translated_df = pd.DataFrame()
    translation = {
        "יצרן": "Brand",
        "שנת הכרזה": "Out Date",
        "מערכת הפעלה": "Operating System",
        "משקל": "Weight",
        "סדרה": "Series",
        "Smartphone": "Smartphone",
        "ממשק הפעלה": "Operating interface",
        "זיכרון RAM": "RAM Capacity",
        "מהירות מעבד": "CPU speed",
        "דור": "CPU generation",
        "גודל מסך": "Screen Size",
        "מספר ליבות": "Number of cores",
        "רזולוציה": "Screen Resolution",
        "צפיפות פיקסלים": "pixels",
        "קצב רענון": "FPS",
        "סוג מסך": "Screen Type",
        "עמידות במים": "Water resistance",
        "צבעים": "Colors",
        "כרטיס זיכרון": "GPU",
        "מצלמה קדמית": "Front camera",
        "מצלמה": "Camera",
        "קורא טביעת אצבע": "Fingerprint reader",
        "WiFi": "WiFi",
        "מחיר מינימלי": "Min price",
        "מחיר מקסימלי": "Max price",
        "NFC": "NFC",
        "Bluetooth": "Bluetooth",
        "קיבולת הסוללה": "Battery capacity",
        "מספר עמוד": "Page number"}
    df.rename(columns=translation,inplace=True)
    for col in translation.values():
        translated_df[col] = df[col]
    return translated_df

<a id='crawling'></a>
<h3>Crawling</h3>
<p>
At this point, we considered different tools for crawling data from zap and ret:
<ul>
<li>BeautifulSoup - a library in python that allows to crawl data via xpath in a simple way</li>
<li>Scrapy - a free open source web-crawling framework written in python, the library allows crawling for data via xpath </li>
<li>Selenum - a tool for automation testing, the selenuim framework is divided into two parts: selenuim sdk and chrome driver. This tool also allows crawling for data via xpath </li>
</ul>

Eventually, we chose BeautifulSoup.
</p>

```python
# ZAP
def zap_data_crawl_all(BASE_URL,filename,page):
    index = page
    df = pd.DataFrame()
    url = f'{BASE_URL}models.aspx?sog=e-cellphone&pageinfo={index}'
    page = requests.get(url,headers = get_fake_user_agent())
    soup = BeautifulSoup(page.text, 'html.parser')
    last_page = int(soup.find('div',attrs={'class':'paging'}).find('select').find_all('option',limit=None)[-1].text)
    for i in range(index,last_page+1):
        a_tags = soup.find('div',attrs={'id':'divSearchResults'}).find_all('a',limit=None)
        phone_id = [tag.get('data-model-id') for tag in a_tags] 
        for i in range(0, len(phone_id)):
            cd = zap_crawl_data(BASE_URL,phone_id[i])
            if(cd is not None):
                cd['מספר עמוד'] = index
                df = pd.concat([df,pd.DataFrame([cd])])
        index += 1
        random_wait()
        url = f'{BASE_URL}models.aspx?sog=e-cellphone&pageinfo={index}'
        page = requests.get(url,headers = get_fake_user_agent())
        soup = BeautifulSoup(page.text, 'html.parser')
    translated_df = translate_names(df)
    translated_df.to_csv(filename, index=False, mode='a', header=False ,encoding = 'utf:"8:"sig')
# RET
def ret_data_crawl_all(BASE_URL,filename,page):
    index = page
    df = pd.DataFrame()
    url = f'{BASE_URL}shop/cellphone?pageindex={index}'
    page = requests.get(url,headers = get_fake_user_agent())
    soup = BeautifulSoup(page.text, 'html.parser')
    last_page =  int(soup.find('nav',attrs={'class':'card-pagination'}).find('div').text.split(' ')[-1])
    for i in range(index,last_page+1):
        phone_divs = soup.find('div',attrs={'class':'product-list'}).find_all('div',attrs={'class':'shop-block'},limit=None)
        phone_id = [div.get('data-id') for div in phone_divs]
        for i in range(0, len(phone_id)):
            cd = ret_crawl_data(BASE_URL,phone_id[i])
            if(cd is not None):
                cd['מספר עמוד'] = index
                df = pd.concat([df,pd.DataFrame([cd])])
        index += 1
        random_wait()
        url = f'{BASE_URL}shop/cellphone?pageindex={index}'
        page = requests.get(url,headers = get_fake_user_agent())
        soup = BeautifulSoup(page.text, 'html.parser')
    translated_df = translate_names(df)
    translated_df.to_csv(filename, index=False,mode='a', header=False,encoding = 'utf:"8:"sig') 
```

In [4]:
ZAP_BASE_URL ="https://www.zap.co.il/"
zap_filename = './data/zap_data.csv'
zap_page = 1

RET_BASE_URL = "https://www.ret.co.il/"
ret_filename = './data/ret_data.csv'
ret_page = 1

```python
zap_data_crawl_all(ZAP_BASE_URL, zap_filename, zap_page)
ret_data_crawl_all(RET_BASE_URL, ret_filename, ret_page)
```

<a id='data_scraped'></a>
<h3>Let's check how much data did we scrape before we start cleaning it (if needed)</h3>

In [5]:
zap_df = pd.read_csv(zap_filename)
ret_df = pd.read_csv(ret_filename)
zap_df['Page number'] = zap_df['Page number'].apply(lambda x: "zap_"+str(x))
ret_df['Page number'] = ret_df['Page number'].apply(lambda x: "ret_"+str(x))
phones_df = pd.concat([zap_df,ret_df],ignore_index=True)
print(f"df shape:{phones_df.shape}")
print(f"data amount before cleaning: {'{:,}'.format(phones_df.shape[0] * phones_df.shape[1])} (rows * cols)")
phones_df.head()

df shape:(2154, 29)
data amount before cleaning: 62,466 (rows * cols)


,Brand,Out Date,Operating System,Weight,Series,Smartphone,Operating interface,RAM Capacity,CPU speed,CPU generation,...,Front camera,Camera,Fingerprint reader,WiFi,Min price,Max price,NFC,Bluetooth,Battery capacity,Page number
0,Samsung,2022,Android,177 גרם,Galaxy S21,סמארטפון,מסך מגע,8GB,1800 2420 2840 MHz,4,...,32 מגה פיקסל,12 +8 + 12 מגה פיקסל,כולל,כולל,"1,444","2,529",כולל,כולל,"4,500 mAh",zap_1
1,Xiaomi,2022,Android,179 גרם,Redmi Note 11,סמארטפון,מסך מגע,6GB,1900 2400 MHz,4,...,13 מגה פיקסל,50 + 8 + 2 + 2 מגה פיקסל,כולל,כולל,680,999,אופציונאלי,5.0,"5,000 mAh",zap_1
2,Samsung,2023,Android,234 גרם,Galaxy S23 Ultra,סמארטפון,מסך מגע,512MB,2800 2800 3360 MHz,5,...,12 מגה פיקסל,200 + 10 + 10 +12 מגה פיקסל,כולל,כולל,"4,489","5,425",אופציונאלי,5.3,"5,000 mAh",zap_1
3,Samsung,2023,Android,234 גרם,Galaxy S23 Ultra,סמארטפון,מסך מגע,12GB,2000 2800 2800 3360 MHz,5,...,12 מגה פיקסל,200 + 10 + 10 +12 מגה פיקסל,כולל,כולל,"4,005","4,925",אופציונאלי,5.3,"5,000 mAh",zap_1
4,Samsung,2022,Android,228 גרם,Galaxy S22,סמארטפון,מסך מגע,12GB,1700 2400 3000 MHz,4,...,40 מגה פיקסל,108 + 10 + 10 + 12 מגה פיקסל,כולל,כולל,"3,418","4,589",כולל,5.2,"5,000 mAh",zap_1


<a id='data_cleaning'></a>
<h2>Data Cleaning</h2>


<a id='data_na_to_real_na'></a>

<h3>Turning Zap's NA's to real NA's</h3>

In [6]:
phones_real_na = phones_df.copy()
for col in phones_real_na:
    phones_real_na.loc[(phones_real_na[col] == 'לא זמין') | (phones_real_na[col] == 'יעודכן בקרוב') | (phones_real_na[col] == 'ללא') | (phones_real_na[col] == "לא רלוונטי"), col] = None 
    phones_real_na.loc[(phones_real_na[col] == 'כולל')| (phones_real_na[col] == 'אופציונאלי'), col] = 1 

<a id='cleaning_and_converting'></a>

<h3>Cleaning and converting to numeric</h3>

In [7]:
def remove_junk(col):
    if(col):
        return col.replace('\u200f','')
    return col

def sum_col(col):
    if col == 'None':
        return None
    col_string = remove_junk(col).split(' ')
    ret_sum = 0
    for i in range(0,len(col_string)):
        ret_sum += float(col_string[i])
    return str(ret_sum)

def multiplay_col(col):
    if(col == 'None'):
        return None
    [num1,num2] = remove_junk(col).lower().split('x')
    return str(float(num1)*float(num2))

def ignore_non_numeric_chars(df,col,ignore_str = r"[^0-9.]",replace=''):
    return df[col].astype(str).str.replace(ignore_str,replace, regex=True).str.strip()

def cleaning_and_converting(df):
    df = df.copy()
    df['Out Date'] = pd.to_numeric(df['Out Date'])
    
    df['Weight'] = pd.to_numeric(ignore_non_numeric_chars(df=df,col='Weight'))
    

    df['Smartphone'] = ignore_non_numeric_chars(df=df,col='Smartphone',ignore_str=r'None',replace='')
    
    df['RAM Capacity'] = pd.to_numeric(ignore_non_numeric_chars(df=df,col='RAM Capacity'))
    
    df['CPU speed'] = ignore_non_numeric_chars(df=df,col='CPU speed',ignore_str=r'MHz')

    df['CPU speed'] = pd.to_numeric(df['CPU speed'].apply(sum_col))

    df['CPU generation'] = pd.to_numeric(ignore_non_numeric_chars(df=df,col='CPU generation',ignore_str=r'[0-9], |None'))
    
    df['Screen Size'] = pd.to_numeric(ignore_non_numeric_chars(df=df,col='Screen Size'))

    df['Number of cores'] = ignore_non_numeric_chars(df=df,col='Number of cores',ignore_str=r'ליבות|\+ |ליבה')
    df['Number of cores'] = ignore_non_numeric_chars(df=df,col='Number of cores',ignore_str=r'אחת',replace='1')
    df['Number of cores'] = ignore_non_numeric_chars(df=df,col='Number of cores',ignore_str=r'\+',replace=' ')
    df['Number of cores'] = ignore_non_numeric_chars(df=df,col='Number of cores',ignore_str=r'Octa-core',replace='8')
    df['Number of cores'] = pd.to_numeric(df['Number of cores'].apply(sum_col))
    
    df['Screen Resolution'] = ignore_non_numeric_chars(df=df,col='Screen Resolution',ignore_str=r'פיקסלים')
    df['Screen Resolution'] = pd.to_numeric(df['Screen Resolution'].apply(multiplay_col)) 
    
    df['pixels'] = ignore_non_numeric_chars(df=df,col='pixels',ignore_str=r'None')
    df['pixels'] = pd.to_numeric(ignore_non_numeric_chars(df=df,col='pixels',ignore_str=r'ppi'))
    
    df['FPS'] = pd.to_numeric(ignore_non_numeric_chars(df=df,col='FPS',ignore_str=r'None|Hz'))
        
    df['Front camera'] = df['Front camera'].apply(remove_junk)
    df['Front camera'] = ignore_non_numeric_chars(df=df,col='Front camera',ignore_str=r'לא כולל מגה פיקסל',replace='0')
    df['Front camera'] = ignore_non_numeric_chars(df=df,col='Front camera',ignore_str=r'כולל מגה פיקסל',replace='1')
    df['Front camera'] = pd.to_numeric(ignore_non_numeric_chars(df=df,col='Front camera',ignore_str=r'\+ |מגה פיקסל').apply(sum_col))
    
    df['Camera'] = ignore_non_numeric_chars(df=df,col='Camera',ignore_str=r'\+ |צגה פיקסל|ים|מ|גה|פיקסל')
    df['Camera'] = ignore_non_numeric_chars(df=df,col='Camera',ignore_str=r" \+",replace=' ')
    df['Camera'] = pd.to_numeric(ignore_non_numeric_chars(df=df,col='Camera',ignore_str=r"\+",replace=' ').apply(sum_col))
    
    
    df['NFC'] = pd.to_numeric(df['NFC'])
    
    df['Bluetooth'] = ignore_non_numeric_chars(df=df,col='Bluetooth',ignore_str=r'לא כולל',replace='0')
    df['Bluetooth'] = pd.to_numeric(ignore_non_numeric_chars(df=df,col='Bluetooth',ignore_str=r'None'))
    
    df['Battery capacity'] = pd.to_numeric(ignore_non_numeric_chars(df=df,col='Battery capacity',ignore_str=r'mAh|,|None'))
    
    df['Min price'] = pd.to_numeric(ignore_non_numeric_chars(df=df,col='Min price',ignore_str=r','))
    df['Max price'] = pd.to_numeric(ignore_non_numeric_chars(df=df,col='Max price',ignore_str=r','))
    # #return df['NFC']
    return df


cleaning_and_converting(phones_real_na).head()


,Brand,Out Date,Operating System,Weight,Series,Smartphone,Operating interface,RAM Capacity,CPU speed,CPU generation,...,Front camera,Camera,Fingerprint reader,WiFi,Min price,Max price,NFC,Bluetooth,Battery capacity,Page number
0,Samsung,2022.0,Android,177.0,Galaxy S21,סמארטפון,מסך מגע,8.0,7060.0,4.0,...,32.0,32.0,1,1,1444,2529,1.0,1.0,4500.0,zap_1
1,Xiaomi,2022.0,Android,179.0,Redmi Note 11,סמארטפון,מסך מגע,6.0,4300.0,4.0,...,13.0,62.0,1,1,680,999,1.0,5.0,5000.0,zap_1
2,Samsung,2023.0,Android,234.0,Galaxy S23 Ultra,סמארטפון,מסך מגע,512.0,8960.0,5.0,...,12.0,232.0,1,1,4489,5425,1.0,5.3,5000.0,zap_1
3,Samsung,2023.0,Android,234.0,Galaxy S23 Ultra,סמארטפון,מסך מגע,12.0,10960.0,5.0,...,12.0,232.0,1,1,4005,4925,1.0,5.3,5000.0,zap_1
4,Samsung,2022.0,Android,228.0,Galaxy S22,סמארטפון,מסך מגע,12.0,7100.0,4.0,...,40.0,140.0,1,1,3418,4589,1.0,5.2,5000.0,zap_1


<a id='categorical_conversion'></a>
<h3>Converting to Categorical</h3>

In [8]:
# ['Brand','Operating System','Operating interface', 'Screen Type', 'Colors','Fingerprint reader','GPU','Fingerprint reader']

categorical_phones = cleaning_and_converting(phones_real_na)
categorical_phones['Water resistance'] = pd.to_numeric(categorical_phones['Water resistance'].apply(lambda col: 0 if col is None else 1))

categorical_phones['NFC'] = pd.to_numeric(categorical_phones['NFC'])

categorical_phones['Smartphone'] = np.where(categorical_phones['Smartphone'].str.strip()  == 'סמארטפון', 1 , 0)

categorical_phones['WiFi'] = pd.to_numeric(np.where(categorical_phones['WiFi'] != 1,0,1))
categorical_phones.head()


,Brand,Out Date,Operating System,Weight,Series,Smartphone,Operating interface,RAM Capacity,CPU speed,CPU generation,...,Front camera,Camera,Fingerprint reader,WiFi,Min price,Max price,NFC,Bluetooth,Battery capacity,Page number
0,Samsung,2022.0,Android,177.0,Galaxy S21,1,מסך מגע,8.0,7060.0,4.0,...,32.0,32.0,1,1,1444,2529,1.0,1.0,4500.0,zap_1
1,Xiaomi,2022.0,Android,179.0,Redmi Note 11,1,מסך מגע,6.0,4300.0,4.0,...,13.0,62.0,1,1,680,999,1.0,5.0,5000.0,zap_1
2,Samsung,2023.0,Android,234.0,Galaxy S23 Ultra,1,מסך מגע,512.0,8960.0,5.0,...,12.0,232.0,1,1,4489,5425,1.0,5.3,5000.0,zap_1
3,Samsung,2023.0,Android,234.0,Galaxy S23 Ultra,1,מסך מגע,12.0,10960.0,5.0,...,12.0,232.0,1,1,4005,4925,1.0,5.3,5000.0,zap_1
4,Samsung,2022.0,Android,228.0,Galaxy S22,1,מסך מגע,12.0,7100.0,4.0,...,40.0,140.0,1,1,3418,4589,1.0,5.2,5000.0,zap_1


<a id='removing_duplicates'></a>
<h3>Removing Duplicates</h3>

In [9]:
laptops_no_dup = phones_real_na[~phones_real_na.duplicated()]

<p>Lets see how many rows did we lose and the NA count:</p>

In [ ]:
laptops_no_dup.info()

<p>It looks like we didn't have any duplicates. so lets continue reviewing the NA's:</p>

In [ ]:
laptops_no_dup.describe()

<p>
it looks like the following coulmns are lacking in data:
<ul>
<strong>
<li>CPU model</li>
<li>Gaming compability</li>
<li>CPU generation</li>
<li>GPU</li>
<li>CPU speed</li>
<li>Memory type</li>
</strong>
</ul>

</p>

<a id='none_values'></a>
<p>
    Let's check how many total books are there without any NA values:
</p>    

In [ ]:
laptops_no_dup.dropna(how='any').info() 

<p>Almost a third missing! thats not good...</p>
<p>First, by looking at the data it seems like:
<br>
<ul>
    <li><strong>Gaming compability</strong></li>
    <li><strong>CPU model</strong></li>
    <li><strong>Optical Drive</strong></li>
    <li><strong>Cellular modem</strong></li>
</ul>
are unnecessary. so, we would like to remove them:</p>

In [ ]:
laptops_no_dup = laptops_no_dup.drop(["CPU model","Gaming Compability",'Optical Drive','Cellular modem' ] , axis = 1) 
laptops_no_dup.dropna(how='any').info()

<p>Not a big improvement... lets try filling-in the rest:</p>


<a id='filling_in_missing_values'></a>
<h3>Filling in missing values<h3>

<h4>First, lets try filling-in <strong>CPU generation</strong>:</h4>
<h5>Mean or Median?</h5>
<p>
    In order to decide in which metric to replace <strong>CPU generation</strong> (mean/median) we want to replace NA values we wanted to base our decision over the distribution of the metric.
</p>

In [ ]:
print(laptops_no_dup['CPU generation'].min())
print(laptops_no_dup['CPU generation'].max())

sns.histplot(laptops_no_dup['CPU generation'])
plt.show()

<p>it seems like 11 is the right choice to fill-in the NA's so we'll go with that</p>

In [ ]:
most_freq_CPU_generation = laptops_no_dup['CPU generation'].mode()[0]
laptops_no_dup['CPU generation'] = laptops_no_dup['CPU generation'].fillna(most_freq_CPU_generation)


<h3>Lets do the same for <strong>CPU speed</strong>:</h3>


In [ ]:
print(laptops_no_dup['CPU speed'].min())
print(laptops_no_dup['CPU speed'].max())

sns.histplot(laptops_no_dup['CPU speed'])
plt.show()

<p>
As can be seen, the distribution can show us that there are not so many outliers.
So we are going to use mean.
</p>

In [ ]:
cpu_speed_mean = laptops_no_dup['CPU speed'].mean()

laptops_no_dup['CPU speed'] = np.where(
    laptops_no_dup['CPU speed'].isnull(),
    cpu_speed_mean,
    laptops_no_dup['CPU speed']
)



<h3>Now, lets try filling-in <strong>GPU</strong> and <strong>Memory type</strong>:</h3>
<p>
  GPU and Memory type are string values, so we'll do the same for them:
</p>

In [ ]:
most_freq_GPU = laptops_no_dup['GPU'].mode()[0]
laptops_no_dup['GPU'] = laptops_no_dup['GPU'].fillna(most_freq_GPU)

most_freq_memory_type = laptops_no_dup['Memory type'].mode()[0]
laptops_no_dup['Memory type'] = laptops_no_dup['Memory type'].fillna(most_freq_memory_type)


<p>lets see how that went for us:</p>

In [ ]:
laptops_no_dup.dropna(how='any').info() 

In [ ]:
laptop_eda = laptops_no_dup.copy().dropna(how='any') 

<p>we saved 500 rows! what a success!</p>


<a id='min_max_to_price'></a>
<h3>Min price, Max price to just Price!</h3>
<p>
    currently, Zap provided us with <strong>Min Price</strong> and <strong>Max Price</strong><br>
    but for our predictions we will need only <strong>Price</strong>.
</p>

In [ ]:
laptop_eda['Price'] = (laptop_eda['Max price'] + laptop_eda['Min price'])/2
laptop_eda['Price'] = (laptop_eda['Price'] - laptop_eda['Price'].min()) / (laptop_eda['Price'].max() - laptop_eda['Price'].min())
laptop_eda = laptop_eda.drop(["Max price" , "Min price" ] , axis = 1) 

<a id='eda'></a>
<h1>EDA</h1>

<p>
    <ul>
        <li>create_heatmap - a function made in order to create and display a heatmap.</li>

In [ ]:
def create_heatmap(numeric_df):
    first_col = numeric_df.pop('Price')
    numeric_df.insert(0, 'Price', first_col)
    sns.heatmap(numeric_df.corr())

<a id='numeric_correlation'></a>
<h3>Numeric fields Correlation to Price</h3>
<p>We would like now to create a heatmap in order to check the correlation of the numeric values on the price:</p>

In [ ]:
laptops_for_heatmap = laptop_eda.select_dtypes(include='number')
create_heatmap(laptops_for_heatmap)

<p>As we can see, the numeric values alone won't help us in our research. but, it seems like we should look into <strong>Ram capacity</strong>.<br>
<strong>Page Number</strong> also has a high correlation to price. We can learn from that, that relevence affects the price as well.</p>

<a id='ram_capacity_distribution'></a>
<h3>Ram Capacity distribution</h3>

In [ ]:
fig, ax = plt.subplots(figsize=(18, 4))
fg = sns.histplot(laptop_eda['RAM Capacity'], ax=ax)
fg.set_title("RAM Capacities Histogram")
fg.set_xlabel("RAM Capacity")
plt.xticks(rotation=90, ha='right', rotation_mode='anchor')
plt.show()

<p>it seems that most laptops use 8 or 16 GB of RAM.</p>

In [ ]:
ram_capacity_counts = laptop_eda['RAM Capacity'].value_counts()
laptops_price_mean_by_ram_capacity = laptop_eda.groupby('RAM Capacity', as_index=False)['Price'].mean()
laptops_price_mean_by_ram_capacity.sort_values(by="Price", inplace=True, ascending=False)

labels = ["RAM Capacity: %s" % i for i in ram_capacity_counts.index]

fig1, ax1 = plt.subplots(figsize=(6, 5))
fig1.subplots_adjust(0.3,0,1,1)

_, _ = ax1.pie(laptops_price_mean_by_ram_capacity['Price'].tolist(), startangle=90)

ax1.axis('equal')

plt.legend(
    loc='upper left',
    labels=['%s, average=%1.1f' % (
        l, (v)) for l, s, v in zip(labels, laptops_price_mean_by_ram_capacity.index, laptops_price_mean_by_ram_capacity['Price'].tolist())],
    prop={'size': 11},
    bbox_to_anchor=(0.0, 1),
    bbox_transform=fig1.transFigure
)

plt.show()

<p>We can learn from this graph that high priced laptops use 8 and 16 GB RAM. due to their distrabution we assume it's because of their populatiry</p>

<a id='brands_distribution'></a>
<h3>Brands distribution</h3>

In [ ]:

fig = plt.figure(figsize=(20, 6)) 
gs = gridspec.GridSpec(1, 2, width_ratios=[3.5, 2.5]) 
ax1 = plt.subplot(gs[0])
ax2 = plt.subplot(gs[1])


#histogram:
fg = sns.histplot(laptop_eda['Brand'], ax=ax1)
fg.set_title("Brands Histogram")
fg.set_xlabel("Brand")
plt.xticks(rotation=90, ha='right', rotation_mode='anchor')

laptopts_by_brand_count = laptop_eda['Brand'].value_counts()

total_laptops = laptop_eda.shape[0]

labels = ["Brand: %s" % i for i in laptopts_by_brand_count.index]

#pie chart:
_, _ = ax2.pie(laptopts_by_brand_count.values, startangle=90)

ax2.axis('equal')

plt.legend(
    loc='upper left',
    labels=['%s, %1.1f%%' % (
        l, (v / total_laptops * 100)) for l, s, v in zip(labels, laptopts_by_brand_count.index, laptopts_by_brand_count.values)],
    prop={'size': 11},
    bbox_to_anchor=(0.0, 1),
    # bbox_transform=fig1.transFigure
)
plt.show()

<p>as can be seen, Lenovo, Asus, Dell and HP are the largest "players" in the market. and Apple and Acer also take a small portion of the market</p><br>
<p>this also checks out with a simple google search:</p>

<img width="50%" height="50%" style="margin-right:5px" src="images/laptop_market_share_1.png"/><img src="images/laptop_market_share_2.png"/>

<a id='mean_price_by_brand_'></a>

<h3>Mean price by brand</h3>

In [ ]:
laptops_price_mean_by_brand = laptop_eda.groupby('Brand', as_index=False)['Price'].mean()
laptops_price_mean_by_brand.sort_values(by="Price", inplace=True, ascending=False)

labels = ["Brand: %s" % i for i in laptopts_by_brand_count.index]

fig1, ax1 = plt.subplots(figsize=(6, 5))
fig1.subplots_adjust(0.3,0,1,1)

_, _ = ax1.pie(laptops_price_mean_by_brand['Price'].tolist(), startangle=90)

ax1.axis('equal')

plt.legend(
    loc='upper left',
    labels=['%s, average=%1.1f' % (
        l, (v)) for l, s, v in zip(labels, laptops_price_mean_by_brand.index, laptops_price_mean_by_brand['Price'].tolist())],
    prop={'size': 11},
    bbox_to_anchor=(0.0, 1),
    bbox_transform=fig1.transFigure
)

plt.show()


<a id='screen_size_distribution'></a>
<h3>Screen size distribution</h3>

In [ ]:
fig, ax = plt.subplots(figsize=(18, 4))
fg = sns.histplot(laptop_eda['Screen Size'], ax=ax)
fg.set_title("Screen sizes Histogram")
fg.set_xlabel("Screen Size")
plt.xticks(rotation=90, ha='right', rotation_mode='anchor')
plt.show()

<p>It's easy to see that 15.6 inches and 14 inches make most of the market. next, we'll check if screen size affects the price.</p>

<a id='mean_price_by_screen_size'></a>

<h3>Mean price by screen size</h3>

In [ ]:
laptopts_by_screen_size_count = laptop_eda['Screen Size'].value_counts()

laptops_price_mean_by_screen_size = laptop_eda.groupby('Screen Size', as_index=False)['Price'].mean()
laptops_price_mean_by_screen_size.sort_values(by="Price", inplace=True, ascending=False)

labels = ["Screen Size: %s" % i for i in laptopts_by_screen_size_count.index]

fig1, ax1 = plt.subplots(figsize=(6, 5))
fig1.subplots_adjust(0.3,0,1,1)

_, _ = ax1.pie(laptops_price_mean_by_screen_size['Price'].tolist(), startangle=90)

ax1.axis('equal')

plt.legend(
    loc='upper left',
    labels=['%s, average=%1.1f' % (
        l, (v)) for l, s, v in zip(labels, laptops_price_mean_by_screen_size.index, laptops_price_mean_by_screen_size['Price'].tolist())],
    prop={'size': 11},
    bbox_to_anchor=(0.0, 1),
    bbox_transform=fig1.transFigure
)

plt.show()


<p>It seems that the screen size does affect the price, but considering that 15.5 and 14 inches make most of the market it's not suprising.</p>

<a id='mean_price_by_gpu'></a>

<h3>Mean price by GPU</h3>

In [ ]:
laptopts_by_gpu_count = laptop_eda['GPU'].value_counts()

laptops_price_mean_by_gpu = laptop_eda.groupby('GPU', as_index=False)['Price'].mean()
laptops_price_mean_by_gpu.sort_values(by="Price", inplace=True, ascending=False)

labels = ["GPU: %s" % i for i in laptopts_by_gpu_count.index]

fig1, ax1 = plt.subplots(figsize=(6, 5))
fig1.subplots_adjust(0.3,0,1,1)

_, _ = ax1.pie(laptops_price_mean_by_gpu['Price'].tolist(), startangle=90)

ax1.axis('equal')

plt.legend(
    loc='upper left',
    labels=['%s, average=%1.1f' % (
        l, (v)) for l, s, v in zip(labels, laptops_price_mean_by_gpu.index, laptops_price_mean_by_gpu['Price'].tolist())],
    prop={'size': 11},
    bbox_to_anchor=(0.0, 1),
    bbox_transform=fig1.transFigure
)

plt.show()


<p>it seems like GPU's affects the price!</p>

<a id='mean_price_by_series'></a>

<h3>Mean price by series</h3>

In [ ]:
laptopts_by_Series_count = laptop_eda['Series'].value_counts()

laptops_price_mean_by_Series = laptop_eda.groupby('Series', as_index=False)['Price'].mean()
laptops_price_mean_by_Series.sort_values(by="Price", inplace=True, ascending=False)

labels = ["Series: %s" % i for i in laptopts_by_Series_count.index]

fig1, ax1 = plt.subplots(figsize=(6, 5))
fig1.subplots_adjust(0.3,0,1,1)

_, _ = ax1.pie(laptops_price_mean_by_Series['Price'].tolist(), startangle=90)

ax1.axis('equal')

plt.legend(
    loc='upper left',
    labels=['%s, average=%1.1f' % (
        l, (v)) for l, s, v in zip(labels, laptops_price_mean_by_Series.index, laptops_price_mean_by_Series['Price'].tolist())],
    prop={'size': 11},
    bbox_to_anchor=(0.0, 1),
    bbox_transform=fig1.transFigure
)

plt.show()


<p>it seems like series affects the price!</p>

<a id='machine_learning'></a>
<h1>Machine Learning</h1>

In [ ]:
laptops_ml = laptop_eda.copy()

<a id='classification'></a>

<h3>Classification of the problem</h3>
<p>We want to predict the price of a laptop based on his specs.<br>
Because what we are predicting is a number, we are facing a suprivesd learning problem called:<br>
<strong>Regression on a continuous variable.</strong><br>
We can solve that problem by using: <strong>Linear Regression.</strong> 
</p>

<a id='ml_helper_functions'></a>

<h3>Helper Functions:</h3>
<p>
<ul>
    <li>train_model - trains the model </li>
    <li>evaluate_performance - evaluates the performance of the model</li>
    <li>split_to_X_and_y - splits the prediction column from the dataframe</li>
    <li>predict - prints a linear regression predicton evaluation of a dataframe using the above functions.</li>
</ul>
</p>


In [ ]:
def train_model(X, y):
    reg = LinearRegression(fit_intercept = False)
    return reg.fit(X, y)

In [ ]:
def evaluate_performance(y,y_predicted):
    return metrics.r2_score(y, y_predicted)

In [ ]:
def split_to_X_and_y(df, target_column):
    return df.drop([target_column],axis = 1), df[target_column]

In [ ]:
def predict(df):
    numeric_df = df.select_dtypes(include='number')
    X, y = split_to_X_and_y(numeric_df,'Price')
    trained_model = train_model(X, y)
    pred_vals = trained_model.predict(X)
    y_pred= pd.Series(pred_vals,index=X.index)
    eval_res = evaluate_performance(y, y_pred)
    print('amount of data:', X.shape[0] * X.shape[1])
    print('prediction:', eval_res)

<a id='predictions'></a>
<h3>Predictions</h3>

<p>First, let's use <strong>Linear Regression</strong> on the dataframe as it to see what's our current prediction:</p>

In [ ]:
predict(laptops_ml)

<p>Lets try to improve that by adding some features:</p>

In [ ]:
def features(df):
    df['Screen Resolution'] = df['Screen Resolution'].apply(lambda s: int(s.split('x')[0]) *int(s.split('x')[1]))
    df['Screen Resolution'] = (df['Screen Resolution'] - df['Screen Resolution'].min()) / (df['Screen Resolution'].max() - df['Screen Resolution'].min())    
    return df  

<p>let's predict again:</p>

In [ ]:
laptops_ml_with_features = features(laptops_ml.copy())
predict(laptops_ml_with_features)

<p>Adding features helped a bit, but not enough.</p>

<p>
<ul>
    <li>categorial_to_numerical - turns a categorical column to numerical using <strong>pd.get_dummies</strong> </li>
</ul>
</p>

In [ ]:
def categorial_to_numerical(df, columns):
    return pd.get_dummies(df, columns=columns)

<p>In the EDA section we saw that <strong>Series</strong> affects the price. lets try converting it to numerical and see if that helps:<br>
(by doing so, we are increasing the amount of data the model is working on, so we won't over use this method)</p>

In [ ]:
laptops_ml_with_categorical = categorial_to_numerical(laptops_ml_with_features.copy(),['Series'])
predict(laptops_ml_with_categorical)

<p>lets try anoter feature to see if it has the same affect:</p>

In [ ]:
laptops_ml_with_categorical = categorial_to_numerical(laptops_ml_with_features.copy(),['GPU'])
predict(laptops_ml_with_categorical)

<p>wow! also a great influnce. not as much as <strong>Series</strong> but also quite big.<br>
Also, a lot less demanding on the amount of data.</p>
<p>Lets combine them:</p>

In [ ]:
laptops_ml_with_categorical = categorial_to_numerical(laptops_ml_with_features.copy(),['GPU','Series'])
predict(laptops_ml_with_categorical)

<p>We'll stop here. this is a great result and the data is getting quite big.<br>
Also, we wouldn't want to overfit the model.</p>

<a id='conclusion'></a>
<h2>Conclusion</h2>

In [ ]:
laptops_conclusion = laptops_ml[['Price','Screen Resolution','RAM Capacity','GPU','Series']].copy()
laptops_conclusion = features(laptops_conclusion)
laptops_conclusion = categorial_to_numerical(laptops_conclusion,['GPU','Series'])
predict(laptops_conclusion)

<p>
    Predicting the price of a laptop with an accurecy of 0.94 requires knowledge of:<br>
    <ul>
        <li>RAM capacity</li>
        <li>series</li>
        <li>GPU</li>
        <li>Screen resolution</li>
    </ul>
    
    Suprisingly, the Brand doesn't affect the price. 
</p>  